# Homework 4 
## Part 1: The $L^2$ Projection in 2D

### This is the solution of:
* Student1
* Student2
* Student3


The same general rules as for Homework 1 applies.

$\newcommand{\dx}{\,\mathrm{d}x}$ 
$\newcommand{\dy}{\,\mathrm{d}y}$
$\newcommand{\dS}{\,\mathrm{d}S}$

In [ ]:
# Make plotted figures interactive
%matplotlib notebook

Import relevant python modules and our own tiny helper modules.

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

Next, we define a surface plotting function for functions defined on triangle meshes.
The triangulation $\mathcal{K}_h$ is given by $n_t$ triangles $\{K\}_{i=0}^{n_t}$ 
and $n_p$ nodes $\{N_i\}_{i=0}^{n_p-1}$. As usual we assume that $\mathcal{K}_h$ is represented by its point matrix $P$ storing in column $j$ the $x$ and $y$ coordinate node $N_j$ of the triangulation, and the connectivity matrix $T$, which in
column $j$ stores the indices of those nodes which comprise triangle $K_j$.

The arguments ```X``` and ```Y``` correspond to the
$0$-th and $1$st row of the point matrix $P$ and ```Z``` is just the array containing
the function values at the nodes. The ```triangles``` argument takes the connectivity matrix $T$. 

In [ ]:
def plot2D(X, Y, Z, triangles, title=''):
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')
    ax.plot_trisurf(X, Y, Z, triangles=triangles.copy(), cmap=cm.viridis, linewidth=0.0)
    ax.set_title(title)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    plt.show()

## Problem 1 (Exploring the meshtools module)

Download $\texttt{meshtools}$ module from Canvas. 
Import it as usual via

In [ ]:
import meshtools as mt

and play around with the implemented functions. In particular, generate and plot 

**a**) a unitsquare mesh with meshsize $h = 1/N$ for $N=1,2,5$.

**b**) 3 rectangle meshes for $[a,b]\times[c,d] = [0,1]\times[2,4]$ for 
3 meshsizes of your choice.

Hint: read the code meshtools.py 

## Problem 2 ($L^2$ projection)

**a**) For a given domain, let $\mathcal{K}_h = \{K_i\}_{i=0}^{n_t-1}$ be a triangulation represent through its point matrix $P$ and its connectivity matrix $T$. As usual, let $V_h$
be the space of continuous, piecewise linear functions defined on $\mathcal{K}_h$

Write a Python function ```assemble_mass_matrix(P, T)``` which assembles the mass
matrix associated with the $L^2$ projection onto $V_h$. Start from the following code outline.

### Code Outline

In [ ]:
def assemble_mass_matrix(P, T):
    # Define constant part M_K here
    M_ref = ...*np.array(....)
    
    # Deduce number of unkowns from dimensions/shape of P
    n_p = ...
    # Deduce number of elements from dimensions of T
    n_t = ...
    
    # Create sparse matrix M
    M = sp.dok_matrix((n_p, n_p))
    
    for  K in range(n_t):
        # Get local to global map from T
        l2g = ...   
        # Get triangle nodes from P
        tri = ...
        # Unpack nodes into N1,N2,N3
        N0,N1,N2 = tri 
        # Compute area of K. 
        # Convince yourself that the following line computes the area |K|
        area=abs(0.5*np.cross(N1-N0,N2-N0))
        # Use area and M_ref to compute M_K
        M_K = ...
        # Instead of 2 loops we can slice out the blocks which
        # corresponds to the entries in l2g by using the funny
        # function ix_ in numpy
        M[np.ix_(l2g, l2g)] += M_K
    
    return M

**b**) Next, write a Python function ```assemble_load_vector(P, T, f)``` 
which assembles the corresponding load vector for a given triangulation $\mathcal{K}_h$ and a right-hand side function f. You shall implement two quadratures (2D Trapezoid and midpoint)to approximate the local element vector

$$
b^K = \int_K f \lambda_i \approx Q_K^{cp}(f\lambda_i) \quad i = 0,1,2
$$

### Code Snippet

In [ ]:
def assemble_load_vector(P, T, f, qr = "midpoint_2d"):
    """ Assembles the load vector """
    
    # Deduce number of unkowns from dimensions/shape of P
    n_p = ...
    # Deduce number of elements from dimensions of T
    n_t = ...
    
    
    # Create and intialize vector
    b = np.zeros(n_p)
    
    # Iterate over all triangles
    for  K in range(n_t):
        l2g = ...   # Get local to global map
        tri = ...  # Get triangle coordinates and compute area
        N0,N1,N2 = tri 
        
        if qr == "midpoint_2d":   
            # 2d midpoint
            # three midpoint coordinates
            N01 = ...
            N12 = ...
            N20 = ...
        
            b_K = ... # an array of three items for the approximation of the integral of
            # f \phi_0, f \phi_1, f \phi_2 in triangle K
        else:
            # 2d Trapezoid
            b_K = # an array of three items for the approximation of the integral of
            # f \phi_0, f \phi_1, f \phi_2 in triangle K
        # Add local contributions to the global load vector
        b[l2g] += b_K
        
    return b

**c)** Finally, we have all the bits and pieces together to compute for a given triangulation $\mathcal{K}_h$ the $L^2$ projection of the analytical function  $f:\Omega \to \mathbb{R}$ defined by

$$
f(x,y) = \sin(2\pi x)\cos(2\pi y),
\quad \Omega = [0,1]^2 \subset \mathbb{R}^2.$$

More specifically, compute $P_h f$ on a series of meshes $\{\mathcal{K}_h\}$
with $h = 1/N$ and $N = 5, 10, 20, 40, 80$.
Plot both $f$, $P_h f$ and the error function $f - P_h f$ for $N = 5, 20, 80$.  Use the 2d Trapezoid rule in the assemble_load_vector.

You can use the following convenient function to plot $f$, $P_h f$ and $f - P_h f$.

### Code Snippet

We define another helper functions which compares two given functions $f_1$ and $f_2$
defined on a triangulation by plotting $\mbox{$f_1$, $f_2$}$ and their difference $f_1 - f_2$ side by side. As for the ```plot_2D``` function defined earlier,
the arguments ```X``` and ```Y``` correspond to the
$0$-th and $1$st row of the point matrix $P$ and ```f1``` and ```f2``` are just the arrays containing the function values of respectively $f_1$ and $f_2$ at the nodes.

You need to use ```import matplotlib.pyplot as plt``` and ```from matplotlib import cm```

In [ ]:
def plot_comparison_2D(X, Y, f1, f2, triangles, title_f1='', title_f2=''):
    fig = plt.figure(figsize=plt.figaspect(0.33))
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    ax.plot_trisurf(X, Y, f1, triangles=triangles.copy(), cmap=cm.viridis, linewidth=0.0)
    ax.set_title(title_f1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')

    # Plot projected function
    ax = fig.add_subplot(1, 3, 2, projection='3d')
    ax.plot_trisurf(X, Y, f2, triangles=triangles.copy(), cmap=cm.viridis, linewidth=0.0)
    ax.set_title(title_f2)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')

    ax = fig.add_subplot(1, 3, 3, projection='3d')
    ax.plot_trisurf(X, Y, f1-f2, triangles=triangles.copy(), cmap=cm.viridis, linewidth=0.0)
    ax.set_title('Difference')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    plt.show()

**d)** Repeat the numerical experiement c) using 
2D midpoint rule in the load vector assembly. 
Describe qualitively the difference and explain why.